**Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

import cris_utils as cu
import data_processing as dp
import conv_deconv_blocks as cdb
import cvae_model as md

**Tensorflow GPU Verification**

In [2]:
print("\n","Versión de tensorflow: " + str(tf.__version__),"\n")
print("Comprobación de la GPU:",tf.config.experimental.list_physical_devices('GPU'),"\n")


 Versión de tensorflow: 2.1.0 

Comprobación de la GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] 



# CVAE

**Model Arquitecture**

In [3]:
N_H = 64
N_W = 64
N_C = 1

enc_convs = [(16,False,cdb.lr_act,3,2),
              (32,False,cdb.lr_act,3,2),
              (64,False,cdb.lr_act,3,2),
             (128,False,cdb.lr_act,3,2),
             (256,False,cdb.lr_act,3,2)]

dec_deconvs = [(256,False,cdb.r_act,3,2),
               (128,False,cdb.r_act,3,2),
               (64,False,cdb.r_act,3,2),
               (32,False,cdb.r_act,3,2),
               (N_C,False,cdb.th_act,3,2)]

n_conv_ls = len(enc_convs)
n_deconv_ls = len(dec_deconvs)

max_layers = cu.max_conv_deconv_layers(N_H)
assert (n_deconv_ls<=max_layers and n_conv_ls<=max_layers), "Incorrect number of layers for generator or discriminator"

config = {
    md.batch_size_k: 16,
    md.latent_dim_k: 32,
    md.adam_alpha_k: 1e-4,
    
    md.data_dir_patt_k:["FVC2006",".png"],
    md.checkpoints_folder_k: "./Checkpoints/",
    md.max_checkpoints_k: 2
}

enc_config = {
    cdb.input_shape_k: (N_H,N_W,N_C),
    cdb.enc_dec_lys_info_k: enc_convs,
    cdb.enc_fin_den_len_k: 2*config[md.latent_dim_k]
}

hw_f_v = N_H/np.power(2,n_deconv_ls)

dec_config = {
    cdb.dec_den_info_k: ([hw_f_v,hw_f_v,256],False,cdb.r_act,(config[md.latent_dim_k],)),
    cdb.enc_dec_lys_info_k: dec_deconvs,
}

for H,W = 64, # deconv layers and # conv layers <= 6
(# conv layers, heigh,width of last enc volume)
(# deconv layers, heigh,width of first dec volume)
[(1, 32), (2, 16), (3, 8), (4, 4), (5, 2), (6, 1)] 



**Data Preparation**

In [4]:
ds_data_dirs = dp.load_process_fp_dataset(config[md.data_dir_patt_k],enc_config[cdb.input_shape_k],config[md.batch_size_k])
#l = list(ds_data_dirs.as_numpy_iterator())
#cu.imshow(l[0][0,:])


Fingerprints Loaded: 1800 from: FVC2006 

Batches Created: 113
shape of batch: (16, 64, 64, 1) 



**Creation of Model**

In [5]:
cvae = md.CVAE(enc_config,dec_config,config)

In [6]:
#print(cvae.encoder.summary())
#print(cvae.decoder.summary())

**Training Configuration**

In [7]:
train_conf = {
    md.num_epochs_k: 2,
    md.types_losses_k: [md.square_loss,md.kl_loss,md.ssim_loss],
    md.alphas_losses_k: [1,0.005,0.5],
    md.dataset_k: ds_data_dirs,
    
    md.create_checkpoints_k: False,
    md.use_latest_checkpoint_k: False,
    md.percent_progress_savings_k: [50,100],
    md.num_images_k: 2,
    md.tensorboard_folder_name_k:"Tensorboard_logs"
}

**Training**

In [8]:
cvae.train(train_conf)

training started
Epoch 1: 8.35 s
Epoch 2: 4.71 s
training finished


**Code Testing**

**Remove Files**

In [2]:
import cris_utils as cu
cu.delete_All("C:/Users/Y50/Documents/Universidad/Tesis/Codigo/Tensorboard_logs/*")

Are you sure you want to delete this? 

C:/Users/Y50/Documents/Universidad/Tesis/Codigo/Tensorboard_logs\20200220-095741
C:/Users/Y50/Documents/Universidad/Tesis/Codigo/Tensorboard_logs\20200220-095846
C:/Users/Y50/Documents/Universidad/Tesis/Codigo/Tensorboard_logs\20200220-100608
Type y or n: y

 Deleted Files
